<a href="https://colab.research.google.com/github/BBB-WU/NLP/blob/BBB-WU-patch-1/NLP_with_BERT_multi_class_text__classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [283]:
pip install transformers

In [284]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [285]:
import pandas as pd

In [286]:
import numpy as np


In [287]:
df = pd.read_csv('/content/train.tsv',sep='\t',header=None)

dt = pd.read_csv('/content/valid.tsv',sep='\t',header=None)

dtt = pd.read_csv('/content/test.tsv',sep='\t',header=None)

In [288]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [289]:
#去掉原来资料库的其他资料，留下label和text
df=df[[1,2]]
df.columns=['Tvalue','Title']

dt=dt[[1,2]]
dt.columns=['Tvalue','Title']

dtt=dtt[[1,2]]
dtt.columns=['Tvalue','Title']

In [290]:
#将六类label分成两类
label_dict = {'barely-true': 0,
 'false': 0,
 'half-true': 0,
 'mostly-true': 1,
 'pants-fire': 0,
 'true': 1}

In [291]:
df['label'] = df.Tvalue.replace(label_dict)
dt['label'] = dt.Tvalue.replace(label_dict)
dtt['label'] = dtt.Tvalue.replace(label_dict)

In [292]:
dt['label'].value_counts()

0    864
1    420
Name: label, dtype: int64

In [293]:
#为了让数据比较平均，选择去掉了两类。

df=df[ ~ df['Tvalue'].str.contains('half-true') ]
df=df[ ~ df['Tvalue'].str.contains('barely-true') ]

dt=dt[ ~ dt['Tvalue'].str.contains('half-true') ]
dt=dt[ ~ dt['Tvalue'].str.contains('barely-true') ]

dtt=dtt[ ~ dtt['Tvalue'].str.contains('barely-true') ]
dtt=dtt[ ~ dtt['Tvalue'].str.contains('barely-true') ]

In [294]:
df.head(10)

,Tvalue,Title,label
0,false,Says the Annies List political group supports ...,0
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",1
3,false,Health care reform legislation is likely to ma...,0
5,true,The Chicago Bears have had more starting quart...,1
9,mostly-true,Says GOP primary opponents Glenn Grothman and ...,1
10,mostly-true,"For the first time in history, the share of th...",1
12,false,When Mitt Romney was governor of Massachusetts...,0
13,mostly-true,The economy bled $24 billion due to the govern...,1
16,true,McCain opposed a requirement that the governme...,1
19,mostly-true,"Almost 100,000 people left Puerto Rico last year.",1


In [295]:
df['label'].value_counts()

1    3638
0    2834
Name: label, dtype: int64

In [296]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [297]:
#利用tokenizer，把句子变成Bert能处理的token形式。

encoded_data_train = tokenizer.batch_encode_plus(
    df.Title.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=32, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    dt.Title.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=32, 
    return_tensors='pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    dt.Title.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=32, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df.label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(dt.label.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(dt.label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [298]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)

dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [299]:
len(dataset_train), len(dataset_val), len(dataset_test)

(6472, 799, 799)

In [300]:
#应用已经写好的model做二元分类。这个model是在bert-base上在加一个nn.linear做分类。

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=2, bias=True)


In [301]:
#把数据分成一个个batch组成的dataloader。

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [302]:
#optimizer应用AdamW，还有一个让learning rate逐渐变小的scheduler。
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-6, 
                  eps=1e-8)

In [303]:
epochs = 120

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)


In [304]:
#加入f1_score来做model预测效果评估。

from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [305]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [306]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [307]:
#用来评估test的结果，会记录下每次预测的结果和正确率。

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [308]:
#这是model train 的部分，每个epoch后，都会把loss和f1_score显示。
#然后，每个epoch， train 完的model都会存下来。在test测试哪个更好。
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    if epoch%5==0:
      torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.6821269666032838
Validation loss: 0.6751371955871582
F1 Score (Weighted): 0.5185427755087415



Epoch 2
Training loss: 0.6620779841991481
Validation loss: 0.6589302968978882
F1 Score (Weighted): 0.6044375484114513



Epoch 3
Training loss: 0.6520691402440001
Validation loss: 0.6498772740364075
F1 Score (Weighted): 0.6179404968181841



Epoch 4
Training loss: 0.6435911015336737
Validation loss: 0.6465421938896179
F1 Score (Weighted): 0.620538116084019



Epoch 5
Training loss: 0.6357499866062785
Validation loss: 0.6457441234588623
F1 Score (Weighted): 0.6224308135900157


...


Epoch 83
Training loss: 0.16246927362577668
Validation loss: 1.0850500869750976
F1 Score (Weighted): 0.6527968773324214



Epoch 84
Training loss: 0.14901621344274488
Validation loss: 1.0991541707515717
F1 Score (Weighted): 0.651752619771053



Epoch 85
Training loss: 0.14131008175298057
Validation loss: 1.105390567779541
F1 Score (Weighted): 0.6437808801839718



Epoch 86
Training loss: 0.14053684928605856
Validation loss: 1.1228185033798217
F1 Score (Weighted): 0.6481459353955488


KeyboardInterrupt: ignored

In [ ]:
#后面就是拿train完的model在test做测试
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

In [338]:

model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_80.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [339]:
_, predictions, true_vals = evaluate(dataloader_test)

In [340]:
accuracy_per_class(predictions, true_vals)

Class: pants-fire
Accuracy: 211/379

Class: true
Accuracy: 309/420



In [342]:

inputs = tokenizer("5G is the cause of covid-19", return_tensors="pt")
inputs.to(device)
  # Batch size 1
outputs = model(**inputs)
logits=outputs[0]
pred = logits.detach().cpu().numpy()
pred=np.argmax(pred, axis=1)
pred

array([0])